In [1]:
# TO DO: 
# Add humidity/wind/etc?  If that route is taken, encapsulate into function
    # minima/maxima should be determined by actual acquired temperatures
    
#gmaps.configure??

#Find out why current query from API is pulling data bout the city itself vs the way it was on the assignment

# Sort hotels by reviews; add price range.

In [2]:
import pandas as pd
import numpy as np
import os
import gmaps
import requests
from pprint import pprint

from config import g_key

gmaps.configure(g_key)

In [3]:
weather_df = pd.read_csv(os.path.join("..","weather_database","WeatherPy_Database.csv"), index_col=0)
weather_df

,City,Country,Lat,Long,Max Temp,Pct Humidity,Pct Cloudiness,Wind Speed,Description
0,Rafai,CF,4.9500,23.9167,81.93,32,72,2.62,broken clouds
1,Saskylakh,RU,71.9167,114.0833,-35.14,100,24,2.95,few clouds
2,Nanortalik,GL,60.1432,-45.2371,22.62,70,7,5.37,clear sky
3,Koroni,GR,36.8000,21.9500,51.51,67,72,10.51,broken clouds
4,Mayo,CA,63.5927,-135.8998,-5.80,84,90,0.00,overcast clouds
5,Rocha,UY,-34.4833,-54.3333,96.76,19,12,10.18,few clouds
6,Saint-philippe,RE,-21.3585,55.7679,78.39,74,93,12.86,overcast clouds
7,Hilo,US,19.7297,-155.0900,69.80,46,90,5.75,moderate rain
8,Vestmannaeyjar,IS,63.4427,-20.2734,32.99,59,20,5.75,few clouds
9,Hobart,AU,-42.8794,147.3294,56.12,82,91,4.61,overcast clouds


In [ ]:
### disallow unreasonable min/max values
min_temp_threshold = 100
max_temp_threshold = 0

#Input strings for repeatability.
min_temp_input_str = f"Please specify a minimum temperature no higher than {min_temp_threshold}: "
not_numeric_str = f"Chosen value is not a number."
threshold_str = f"Chosen value is outside specified threshold."
min_higher_than_max_str = f"Your max temperature is higher than your min temperature."

min_temp = input(min_temp_input_str)

# iterate on input until a valid value is given
while min_temp.isnumeric() == False or float(min_temp) > min_temp_threshold:
    if min_temp.isnumeric() == False:
        print(not_numeric_str)
    elif float(min_temp) > min_temp_threshold:
        print(threshold_str)
    min_temp = input(min_temp_input_str)
    
# Replace max threshold with the min value (if higher)
max_temp_threshold = max(float(min_temp),float(max_temp_threshold))
max_temp_input_str = f"Please specify a maximum temperature no lower than {int(max_temp_threshold)}: "

max_temp = input(max_temp_input_str)

# iterate on input until a valid value is given
while max_temp.isnumeric() == False or float(max_temp) < max_temp_threshold:
    if max_temp.isnumeric() == False:
        print(not_numeric_str)
    elif float(max_temp) < max_temp_threshold:
        print(threshold_str)
    max_temp = input(max_temp_input_str)
    
# Generate data frame from preferred temperatures and remove blanks
vacation_cities_df = weather_df.loc[(weather_df["Max Temp"] >= float(min_temp)) & (weather_df["Max Temp"] <= float(max_temp))].dropna()

print("------------------------")
print("Cities identified based on search criteria:")
vacation_cities_df

In [ ]:
gmaps_base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

vacation_cities_df["Hotel Name"] = ""

for index, row in vacation_cities_df.iterrows():
    location = f'{vacation_cities_df.loc[index]["Lat"]},{vacation_cities_df.loc[index]["Long"]}'

    params = {'key':g_key,
              'location':location,
              'radius':5000,
              'type':'logding'
            }
    
    try:
        gmaps_request = requests.get(url=gmaps_base_url, params=params)
        gmaps_json = gmaps_request.json()
        
        #results[0] just yields city name.
        hotel_name = gmaps_json['results'][1]['name']
        
        print(f"Identified {hotel_name} in {row['City']}, {row['Country']}")        
        vacation_cities_df.loc[index, "Hotel Name"] = hotel_name
    except KeyError:
        print(f"No hotels found for {row['City']}, {row['Country']}")
        
vacation_cities_df = vacation_cities_df.dropna()
vacation_cities_df

In [ ]:
# reset my index before you forget.  Also don't hardcode this csv.
vacation_cities_df.to_csv("WeatherPy_vacation.csv")

In [ ]:
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Today's Weather</dt><dd>{Max Temp} and {Description}</dd>
<dt>Hotel Name</dt><dd>{Hotel Name}
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in vacation_cities_df.iterrows()]

coordinates_df = vacation_cities_df[["Lat","Long"]]
coordinates_df

fig = gmaps.figure(center=(0.0, 0.0), 
                   zoom_level=1.5,
                   layout = {'width': '800px',
                             'height': '500px'})

markers = gmaps.marker_layer(locations=coordinates_df, info_box_content=hotel_info)
fig.add_layer(markers)

## Call the figure
fig